## 2021 Fall "EE488 : Quantum and Solid-State Physics for Semiconductor Devices"


## Tight binding model: Single-walled carbon nanotubes

Writer: Ryong-Gyu, Lee (KAIST EE, Yong-Hoon Kim Group)  
2021.10.12 (revised)  

**References**:  

1. Electronic band structure (zone-folding scheme):
http://demonstrations.wolfram.com/ElectronicBandStructureOfASingleWalledCarbonNanotubeByTheZon/

2. Analysis: metallicity condition, electronic DOS, & bandgap:
http://demonstrations.wolfram.com/MetallicityConditionAndElectronicDensityOfStatesInAchiralSin/


### Electronic band structure of a single-walled carbon nanotube by the zone-folding method

In [13]:
%matplotlib inline
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def nanotube_band(n=4, m=0, e=0, s=0, t=-2, mode='2D'):
    # Analytical expression for graphene tight - binding electronic dispersion relation   
    def _Ek(e, s, t, kx, ky):    
        f  = np.exp(1j*kx*acc) + 2 * np.exp(-1j*kx*acc/2) * np.cos(ky*a0/2)
        ff = np.exp(-1j*kx*acc) + 2 * np.exp(1j*kx*acc/2) * np.cos(ky*a0/2)
        w  = np.sqrt(f * ff)
        Ev = np.real((e+t*w) / (1+s*w))
        Ec = np.real((e-t*w) / (1-s*w))

        return Ev, Ec
    
    # Rotation function for cutting line
    def _rotate(vector, angle):
        x = vector[0]*np.cos(angle)-vector[1]*np.sin(angle)
        y = vector[0]*np.sin(angle)+vector[1]*np.cos(angle)
        return np.array([x,y])

    # Cutting lines applied to graphene tight - binding electronic structure : zone - folding method
    def _plot1D():
  
        fig =plt.figure(figsize=(4,4))
    
        # k path
        kz = np.linspace(-np.pi/lucaxis,np.pi/lucaxis,60)
        plt.xticks([kz[0],0,kz[-1]],[r'$-\frac{\pi}{T}$',r'$0$',r'$\frac{\pi}{T}$'])
        plt.ylabel(r'$Energy [eV]$')

        for i in range(int(nc)):
            kx = i*K1nt[0] + kz*K2nt[0]/K2norm
            ky = i*K1nt[1] + kz*K2nt[1]/K2norm
            Ev, Ec = _Ek(e, s, t, kx, ky)
            plt.plot(kz,Ev)
            plt.plot(kz,Ec)

        plt.tight_layout()
        plt.show()

    # Energy Contour Plot for 2d Brillouin zone
    def _plot2D():

        fig =plt.figure(figsize=(6,4))
        
        X = np.linspace(-2.7, 2.7, 60)
        Y = np.linspace(-2.7, 2.7, 60)
        Kx, Ky = np.meshgrid(X, Y) 

        # calculate energy dispersion
        Ev, Ec = _Ek(e, s, t, Kx, Ky)
        
        # plot BZ boundary
        plt.plot(K12[:,0], K12[:,1],'k', linewidth = 2)
        plt.plot(K23[:,0], K23[:,1],'k', linewidth = 2)
        plt.plot(K34[:,0], K34[:,1],'k', linewidth = 2)
        plt.plot(K45[:,0], K45[:,1],'k', linewidth = 2)
        plt.plot(K56[:,0], K56[:,1],'k', linewidth = 2)
        plt.plot(K61[:,0], K61[:,1],'k', linewidth = 2)
        
        # cut line
        for i in range(len(cutlineset)):
            plt.plot(cutlineset[i][:,0], cutlineset[i][:,1],'r', linewidth = 2)
        plt.xlabel(r'$K_{x}$')
        plt.ylabel(r'$K_{y}$')
        plt.xlim([-2.7,2.7])
        plt.ylim([-2.7,2.7])
        # Energy Contour Plot
        plt.contourf(Kx, Ky, Ev, levels=8, cmap="RdBu_r")
        plt.tight_layout()
        plt.show()

    # Parameters
    acc = 1.42             # Carbon-Carbon bond length
    a0  = np.sqrt(3) * acc # graphene lattice parameter
    a1  = a0 * np.array([np.sqrt(3)/2, 1/2])    # real unit vectors of graphene lattice 
    a2  = a0 * np.array([1/np.sqrt(3), -1/2])
    b1  = (2*np.pi/a0) * np.array([1/np.sqrt(3),1])  # reciprocal unit vectors of graphene
    b2  = (2*np.pi/a0) * np.array([1/np.sqrt(3),-1])

    # Nanotube Brillouin zone
    BZKx = 2*np.pi / (np.sqrt(3)*a0)
    BZKy = 2*np.pi / (3*a0)
    Gamma = np.array([0,0])
    M1    = np.array([BZKx,0])
    K1    = np.array([BZKx,-BZKy])
    K2    = np.array([BZKx,BZKy])
    K3    = np.array([0,2*BZKy])
    K4    = np.array([-BZKx,BZKy])
    K5    = np.array([-BZKx,-BZKy])
    K6    = np.array([0,-2*BZKy])
    
    # Hexagonal BZ
    K12 = np.linspace(K1,K2,30)
    K23 = np.linspace(K2,K3,30)
    K34 = np.linspace(K3,K4,30)
    K45 = np.linspace(K4,K5,30)
    K56 = np.linspace(K5,K6,30)
    K61 = np.linspace(K6,K1,30)
    BZKz = np.zeros(30)
    
    # Highest common divisor 
    d = np.gcd(n,m)
    dr = np.gcd(2*n+m,2*m+n)

    # Nanotube chiral angle
    chiralangle = np.arccos((2*n+m)/(2*np.sqrt(n**2+m**2+n*m)))

    # Nanotube axial unit vector
    t1 = (2*m+n) / dr ; t2 = -(2*n+m) / dr

    # Number of grapghene cells inside the nanotube unit cell
    nc = t1*m - t2*n

    # Length of nanotube unit cell along diameter
    lucdiam = np.linalg.norm(n*a1 + m*a2)

    # Length of nanotube unit cell along tube axis
    lucaxis = np.linalg.norm(t1*a1 + t2*a2)

    #Nanotube reciprocal space unit vectors and norms
    K1nt = (t1*b2-t2*b1) / nc ; K2nt = (m*b1-n*b2) / nc
    K1norm = np.linalg.norm(K1nt) ; K2norm = np.linalg.norm(K2nt)

    # Cutting - lines
    cutlineset = []
    for i in range(int(nc)):
        u = -nc/2 + 1 + i
        line_i = _rotate(np.array([-K2norm/2, u*K1norm]),chiralangle)
        line_f = _rotate(np.array([K2norm/2, u*K1norm]),chiralangle)
        line = np.linspace(line_i,line_f,30)
        cutlineset.append(line)
        
    if mode=='1D':
        _plot1D()
    elif mode=='2D':
        _plot2D()
        
    plt.close()

if __name__=="__main__":
    
    import ipywidgets as widgets
    from ipywidgets import interact, fixed
    
    @interact(n=(4,50,1))
    def interactive(n=4):
        
        widgets.interact(nanotube_band,
                         n = fixed(n),
                         m = (0,n,1),
                         e = (-1,1,0.1),
                         s = (-0.2,0.2,0.01),
                         t = (-4,0,0.1),
                         mode = ["1D","2D"]
                        )

         

interactive(children=(IntSlider(value=4, description='n', max=50, min=4), Output()), _dom_classes=('widget-int…

### Discussion

- Referring to CNT Brillouin zone, check that the chiral angle ($\theta$) between chiral vector ($C_h$) and $a_1$  follows the relation,

$$
\tan\theta = \frac{\sqrt{3}n}{2m+n}
$$

- For armchair (n,n) and zigzag (n,0) CNTs, how many bands appear in the zone-folding band structures? And you can aslo check which bands correspond to degenate states  
**Hint:** How many carbon atoms are included within the CNT unit cell?

- Check which bands give the dispersionless energy bands for the zigzag (n,0) CNTs  
**Hint:** Check the CNT Brillouin zone and corresponding band dispersion

Check the reference: https://doi.org/10.1016/0008-6223(95)00017-8

### Metallicity condition and electronic density of states in achiral single-walled carbon nanotubes

In [9]:
%matplotlib inline
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


def nanotube_dos(structure, n=4, t=2, e=0, erange=1):

    # Green' s function poles for zigzag nanotubes
    def polecoszz(w, g, jj, mm):
        return (w**2/g**2 - 1 - 4*(np.cos(np.pi*jj/mm))**2) / (4*np.cos(np.pi*jj/mm))

    # Green' s function poles for armchair nanotubes
    def polecosaplus(w, g, jj, mm):
        return -1/2 * (np.cos(np.pi*jj/mm) + np.sqrt(w**2/g**2-(np.sin(jj/mm*np.pi))**2))

    def polecosaminus(w, g, jj, mm):
        return -1/2 * (np.cos(np.pi*jj/mm) - np.sqrt(w**2/g**2-(np.sin(jj/mm*np.pi))**2))
    
    # plot density of states
    def _plot():

        fig =plt.figure(figsize=(6,4))
        
        # Energy range (onsite term e = 0)
        ee = np.linspace(-erange*t, erange*t, 10001)
        if m == 0:
            for u in range(1,int(nc)+1):
                d = np.imag(1j*(ee-e) / (nc*t**2*(np.cos(u/n*np.pi)*np.sin(np.arccos(polecoszz(ee-e,t,u,n))))))
                plt.fill_between(ee, d, color = 'orange', alpha=1)

        else:
            for u in range(1,int(nc)+1):
                d1 = np.imag(1j*(ee-e) / (nc*t**2*(np.cos(u/n*np.pi)*np.sin(np.arccos(polecosaplus(ee-e,t,u,n))) +
                                                2 * polecosaplus(ee-e,t,u,n) * np.sin(np.arccos(polecosaplus(ee-e,t,u,n)))))) 
                d2 = np.imag(1j*(ee-e) / (nc*t**2*(np.cos(u/n*np.pi)*np.sin(np.arccos(polecosaminus(ee-e,t,u,n))) +
                                                2 * polecosaminus(ee-e,t,u,n) * np.sin(np.arccos(polecosaminus(ee-e,t,u,n))))))
                d3 = d1+d2
                plt.fill_between(ee, d1, color = 'orange', alpha=1)
                plt.fill_between(ee, d2, color = 'orange', alpha=1)

        plt.xlabel(r'$E-E_F [eV]$')
        plt.ylabel(r'$DOS [states/eV \times atom]$')   
        plt.ylim(0,3)
        plt.tight_layout()
        plt.show()
    
    # define the CNT structure
    if structure=="armchair(n,n)":
        m = n
    elif structure=="zigzag(n,0)":
        m = 0

    # parameters
    acc = 1.42             # Carbon-Carbon bond length
    a0  = np.sqrt(3) * acc # graphene lattice parameter
    a1  = a0 * np.array([np.sqrt(3)/2, 1/2])    # real unit vectors of graphene lattice 
    a2  = a0 * np.array([1/np.sqrt(3), -1/2])
    b1  = (2*np.pi/a0) * np.array([1/np.sqrt(3),1])  # reciprocal unit vectors of graphene
    b2  = (2*np.pi/a0) * np.array([1/np.sqrt(3),-1])
    
    # highest common divisor 
    d = np.gcd(n,m)
    dr = np.gcd(2*n+m,2*m+n)

    # nanotube chiral angle
    chiralangle = np.arccos((2*n+m)/(2*np.sqrt(n**2+m**2+n*m)))

    # nanotube axial unit vector
    t1 = (2*m+n) / dr ; t2 = -(2*n+m) / dr

    # number of grapghene cells inside the nanotube unit cell
    nc = t1*m - t2*n

    # length of nanotube unit cell along diameter
    lucdiam = np.linalg.norm(n*a1 + m*a2)

    # length of nanotube unit cell along tube axis
    lucaxis = np.linalg.norm(t1*a1 + t2*a2)

    # calculate and plot density of states
    _plot()

if __name__=="__main__":
    
    np.seterr(all="ignore") # skip error
    import ipywidgets as widgets
    widgets.interact(nanotube_dos,
                     structure=["armchair(n,n)","zigzag(n,0)"],
                     t = (0,3,0.1),
                     e = (-1,1,0.1),
                     n = (4,10,1),
                     erange = (1,5,1),
                    )


interactive(children=(Dropdown(description='structure', options=('armchair(n,n)', 'zigzag(n,0)'), value='armch…

### Discussion

- Check metallicity condition for the zigzag (n,0) CNTs  
- Check metallicity condition for the armchair (n,n) CNTs  

**Hint:** you can also find metallicity conditions from the CNT Brillouin zone and corresponding band dispersion